In [ ]:
import importlib, sys
from utils.utils import ThCall

if mod := sys.modules.get("download_census_data"): importlib.reload(mod)
import download_census_data

#engine().execute("drop table census.dec2020pl_p2_tract")

#ds = CensusApiDataset("2010/dec/sf1")


In [ ]:

ThCall(download_census_data.display_storage)

for dataset in [
    #"2020/dec/pl",
    #"2010/dec/sf1", 
    #"2010/dec/sf2",
    #"2000/dec/sf1",
    #"2000/dec/sf2", 
    #"2000/dec/sf3",
    #"2020/acs/acs5"
    #"2021/acs/acs5",
    # See https://www.census.gov/data/developers/data-sets/decennial-census.html
    "2020/dec/dhc", # Demographic and Housing Characteristics File (DHC)
    ]:
    ds = download_census_data.CensusApiDataset(dataset)
    ds.download_tables_geo_levels(nthreads=15)

In [ ]:
ds = download_census_data.CensusApiDataset("2021/acs/acs5")
#df = ds.api_get({'get': 'group(B07404APR)', 'for': 'state:*'})
table_name = 'B98013'
counties = ds.api_get({'get': f'group({table_name})', 'for': 'county:*'})
variables = list(ds.tables[table_name].keys())    

#counties = ds.api_get({'get': 'group(B07404EPR)', 'for': 'county:*'})
#variables = ['B07404EPR_001E', "B07404EPR_005E"]
county_has_data = ~counties[variables].isna().any(axis=1)
sorted(counties[county_has_data]['state'].unique())

In [ ]:
states = ds.api_get({'get': f'group({table_name})', 'for': 'us:*'})
states


In [ ]:

#census_api_get('https://api.census.gov/data/2021/acs/acs5', {'get': 'group(B07001PR)', 'for': 'state:*'})

# import pandas as pd
# pd.set_option('display.max_rows', None)

# ds.api_get({'get': 'group(B07404GPR)', 'for': 'county:*', 'in': 'state:72'})


In [ ]:
# for table in engine().list_tables("census"):
#     if table.startswith("dec2020"):
#         engine().execute("drop table census." + table)
#     else:
#         print(table)

In [ ]:
#engine().execute_returning_df('select "POPGROUP" from census.dec2010sf2_hct1_state limit 10')